## word embedding의 의미??

learned representation of words를 의미한다.

- similar meanings => similar representation 전제에서 출발.

Dense, low dimensional representations 형태로, NN에 적용시켜 학습하기 편하다. (원 핫 인코딩에 비해서)

(특정 word가 나타났는지 여부 확인 + 해당 단어의 의미를 encapsulate할 수 있는 거라고 보면 된다)

예컨대 20000개 단어를 one_hot 하면 input은 20000개일 거고, 대부분의 값은 0일 것이다. 이거 대신 사용하려는 것.

ex) this is awful movie / this is terrible movie. 두 개의 단어는 형용사만 제외하면 동일한 의미이다.

이 경우는 contextual operation을 할 수 있다. by performing vector operations on these learned representations.

King - men + women = Queen...


워드 임베딩 알고리즘의 종류
- embedding layer -> Neural network의 한 레이어를 아예 embedding으로 할애하는 것
- word2vec -> predicts words in similar context. 
- glove -> similar context의 word count를 바탕으로 한 알고리즘





In [1]:
import gensim

In [2]:
def load_data():
    reviewsFile = open("./Hands-on-NLP-with-PyTorch-master/data/reviews.txt",'r')
    reviews = list(map(lambda x:x[:-1],reviewsFile.readlines()))
    reviewsFile.close()

    labelsFile = open("./Hands-on-NLP-with-PyTorch-master/data/labels.txt",'r')
    labels = list(map(lambda x:x[:-1],labelsFile.readlines()))
    labelsFile.close()
    
    return reviews,labels

reviews, labels = load_data()

In [3]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("\w+\'?\w+|\w+")

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from spacy.lang.en.stop_words import STOP_WORDS

exceptionStopWords = {
    'again',
    'against',
    'ain',
    'almost',
    'among',
    'amongst',
    'amount',
    'anyhow',
    'anyway',
    'aren',
    "aren't",
    'below',
    'bottom',
    'but',
    'cannot',
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'don',
    "don't",
    'done',
    'down',
    'except',
    'few',
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'however',
    'isn',
    "isn't",
    'least',
    'mightn',
    "mightn't",
    'move',
    'much',
    'must',
    'mustn',
    "mustn't",
    'needn',
    "needn't",
    'neither',
    'never',
    'nevertheless',
    'no',
    'nobody',
    'none',
    'noone',
    'nor',
    'not',
    'nothing',
    'should',
    "should've",
    'shouldn',
    "shouldn't",
    'too',
    'top',
    'up',
    'wasn',
    "wasn't",
    'well',
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't",
}

stop_words = set(stop_words).union(STOP_WORDS)

final_stop_words = stop_words-exceptionStopWords

import spacy
nlp = spacy.load("en",disable=['parser', 'tagger', 'ner'])

def make_token(review):
    return tokenizer.tokenize(str(review))

def remove_stopwords(review):
    return [token for token in review if token not in final_stop_words]

def lemmatization(review):
    lemma_result = []
    
    for words in review:
        doc = nlp(words)
        for token in doc:
            lemma_result.append(token.lemma_)
    return lemma_result

def pipeline(review):
    review = make_token(review)
    review = remove_stopwords(review)
    return lemmatization(review)

# %%time
reviews = list(map(lambda review: pipeline(review),reviews))

In [6]:
reviews[:2]
# base form이며, stopwords는 제거된 상태.

[['bromwell',
  'high',
  'cartoon',
  'comedy',
  'run',
  'time',
  'program',
  'school',
  'life',
  'teacher',
  'year',
  'teach',
  'profession',
  'lead',
  'believe',
  'bromwell',
  'high',
  'satire',
  'much',
  'close',
  'reality',
  'teacher',
  'scramble',
  'survive',
  'financially',
  'insightful',
  'student',
  'right',
  'pathetic',
  'teacher',
  'pomp',
  'pettiness',
  'situation',
  'remind',
  'school',
  'know',
  'student',
  'see',
  'episode',
  'student',
  'repeatedly',
  'try',
  'burn',
  'down',
  'school',
  'immediately',
  'recall',
  'high',
  'classic',
  'line',
  'inspector',
  'sack',
  'teacher',
  'student',
  'welcome',
  'bromwell',
  'high',
  'expect',
  'adult',
  'age',
  'think',
  'bromwell',
  'high',
  'far',
  'fetch',
  'pity',
  'isn'],
 ['story',
  'man',
  'unnatural',
  'feeling',
  'pig',
  'start',
  'open',
  'scene',
  'terrific',
  'example',
  'absurd',
  'comedy',
  'formal',
  'orchestra',
  'audience',
  'turn',
  '

In [7]:
from gensim.models import Word2Vec

In [8]:
embedding_dim = 100
# 이 값은 보통 vocab size & text couples에 따라 다르다.

In [9]:
model = Word2Vec(reviews, size = embedding_dim, window = 3, min_count = 3, workers = 4)
# window = N of words that the embedding has to consider for a given words.
# min_count는 3. 최소 3번은 등장한 단어부터 model에 포함된다는 소리
# train을 마치면, model을 삭제하는 걸 추천한다고 함. 메모리 소모가 많기 때문이라고.
# embedding에 필요한 모든 정보를 가지고 있는 model.wv를 쓴다.

In [10]:
word_vectors = model.wv
del model

In [12]:
len(word_vectors.vocab)

28165

word embedding이 찾아 준, similar words 조합을 찾아보자


In [13]:
word_vectors.similar_by_word(word='good', topn=5)
# 상위 5개 조합. 비슷해 보이는 의미라고 유추 가능하다.

[('decent', 0.7098484039306641),
 ('alright', 0.670569121837616),
 ('darn', 0.6590393781661987),
 ('okay', 0.6483892202377319),
 ('great', 0.6436715126037598)]

In [14]:
word_vectors.similar_by_word(word='bad', topn=5)
# 상위 5개 조합. 비슷해 보이는 의미라고 유추 가능하다.

[('terrible', 0.7132728099822998),
 ('horrible', 0.7132444381713867),
 ('suck', 0.6856053471565247),
 ('lousy', 0.679572343826294),
 ('awful', 0.6717656850814819)]

In [15]:
word_vectors.most_similar(positive='bad', topn=5)

[('terrible', 0.7132728099822998),
 ('horrible', 0.7132444381713867),
 ('suck', 0.6856053471565247),
 ('lousy', 0.679572343826294),
 ('awful', 0.6717656850814819)]

In [16]:
word_vectors.similarity('good','bad')
# 두 단어가 similar context에서 등장할 경우 높고, 아닐 경우 낮은 값이 나온다
# 다시말해 this is a good movie / this is a bad movie와 같이 유사한 형태로 많이 등장하는 단어라는 뜻이고

0.5713808953482875

In [17]:
word_vectors.similarity('good', 'be')
# when they don't appear in simliar context.
# this is a good movie와 같은 문장 맥락에서 be라는 단어는 잘 안 쓰였다는 의미다.

0.29746172755519396

In [18]:
word_vectors.similar_by_word('sad',topn=5)

[('depress', 0.7849045991897583),
 ('cry', 0.7138973474502563),
 ('heartwarming', 0.6890783905982971),
 ('happy', 0.679260790348053),
 ('scary', 0.6663157939910889)]

In [19]:
# contextual relationship btwn words
# king - men + women = queen 같은
word_vectors.most_similar(negative = ['bad'], positive=['decent'],topn=5)
# positive = needs to be added, negative= needs to be substracted
# contextual relationship의 workout을 이런 식으로 확인할 수 있다고 함

[('solid', 0.3796919882297516),
 ('fine', 0.3740949034690857),
 ('nicely', 0.3674663305282593),
 ('support', 0.3629467189311981),
 ('splendid', 0.36054667830467224)]

## Pretrained Embedding models

- word embedding = statistic method to learn standalone word embeddings from a text corpus. 2 Different types of models.

2 dif types {
    continuous bag of words : current word is predicted from its surroundings.
    continuous skip gram model : surrounding words are predicted from a current words.
}
두 개의 모델을 바탕으로 만들어지는 게 word embedding

pretrained 모델을 쓸 수 있다고 함.

``from gensim.models import KeyedVectors``

``model = KeyedVectors.load_word2vec_format('data/GoogleGoogleNews-vectors-negative300.bin/', binary=True)``

구글 뉴스데이터를 바탕으로 한 값들. 특정 디렉토리에 다운로드해 저장한 다음 필요할 때 꺼내쓰는 게 일반적이다.



# RNN과 LSTM 도입

RNN - designed for persistence of information. time series와 sequence of data 특화형. 음악을 듣는 거에 비유함.

RNN은 memory capability가 있다. takes into account all the information that it has seen so far, to generate the next output.

network loop allows for the persistence of information here.

Variable size input / output을 받는다.

perform variable number of computation (traditional network는 perform fixed number of computation)

information persistence.

Output = history of all inputs를 토대로 만들어진 결과물.

one-to-many, many-to-one, many-to-many

video anaylsis, speech recognition, language modeling, translations, time series analysis, text classification

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
SEED = 2222

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True
# 재현성 구현을 위해 시드 고정

In [22]:
# text를 받아서 corresponding index로 변환하는 함수가 필요함. 
# words in Corpus를 corresponding index로. 이 index를 RNN에 학습 대상으로 넣는 것
def word2idx(embedding_model, review):
    index_review = []
    for word in review:
        try:
            index_review.append(embedding_model.vocab[word].index)
        except:
            pass
    return torch.tensor(index_review)


In [23]:
padding_value = len(word_vectors.index2word)

In [27]:
# review의 길이를 맞춰주기 위해서, 짧은 review에는 padding_value라는 값을 넣어준다고 함.
# 왜 그러는지 강의에서 뭐라 하긴 하는데 뭔소린지 모르겠음
padding_value

28165

In [28]:
index_review = list(map(lambda review: word2idx(word_vectors, review),reviews))

In [29]:
index_review

[tensor([16210,   143,   628,    90,   123,     7,  1348,   251,    43,  1160,
            40,  1162,  4497,   119,   116, 16210,   143,  1551,    19,   229,
           432,  1160,  9480,   895,  8041,  4729,   625,    98,   981,  1160,
         15367,   397,   559,   251,    22,   625,    13,   142,   625,  3047,
            46,   761,    86,   251,   997,  1518,   143,   203,    99,  2348,
          5165,  1160,   625,  1587, 16210,   143,   145,   532,   255,    14,
         16210,   143,   120,  2747,  1723,   112]),
 tensor([   11,    35,  5988,  1146,  2628,    77,   204,    16,  1060,   287,
          1416,    90,  8393,  5835,   140,    85,  1747,   883,  2377,   688,
          1266,   335,   361,  1416,     7,    12,   596,  1026,   664,    37,
            36,  1222,   739,    52,    85, 10618,   269,  1474,   483,  6571,
          1428,   379,    15,    14,     5,   469,   517,    18, 21804, 21805,
           517,    62,  2778,  9009, 12751,  6363,    13,  2758]),
 tensor([17

In [30]:
# 1st layer = embedding layer
embedding_weights = torch.Tensor(word_vectors.vectors)

In [31]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,embedding_weights):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_weights)
        self.rnn = nn.RNN(embedding_dim, hidden_dim) # input, output
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x, text_lengths):
        # x = [sent length , batch size]
        # text length는 length of individual reviews within each batch that we pass into our model.
        # each review는 padded such that all reviews within each batch is of same length.

        # 가장 긴 review의 length에 맞춰야 한다. 하지만 padding으로 들어간 데이터는 모델 성능에 영향을 미쳐선 안 된다.
        # pytorch에서는 이 기능을 pack_padded_sequence라는 함수로 제공한다고 함. 
        # (packs together these padded values and internally handles for us)
        
        # 즉, 모델에 input으로 넣을 x는 [sentence length, batch size]
        embedded = self.embedding(x)
        # embedding을 통과한 결과는 [sentence length, batch size, embedding dimension]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, hidden = self.rnn(packed_embedded)
        #[sentence length,batch size, hidden dim],[1,batch size,hidden dim]

        # undo pack_padded sequence 함수라고 함
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        return self.fc(hidden.squeeze(0))

In [32]:
INPUT_DIM = padding_value
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

In [34]:
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, embedding_weights)

In [35]:
model

RNN(
  (embedding): Embedding(28165, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [36]:
optimizer = optim.SGD(model.parameters(), lr = 1e-3)

In [38]:
loss = nn.BCEWithLogitsLoss() # performing sigmoid보다 numerically stable하다고 한다.

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:
device

device(type='cpu')

# Train / test / Validation 데이터로 구분

In [42]:
from sklearn.model_selection import train_test_split
labels[:5]

['positive', 'negative', 'positive', 'negative', 'positive']

In [43]:
labels = [0 if label=='negative' else 1 for label in labels]

In [45]:
labels[:5]

[1, 0, 1, 0, 1]

In [53]:
X_train, X_test, y_train, y_test = train_test_split(index_review, labels, test_size = .2)

In [54]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .2)

In [58]:
print(len(X_train), len(X_test), len(X_val))
print(len(y_train), len(y_test), len(y_val))

16000 5000 4000
16000 5000 4000


In [ ]:
batch_size = 128
import numpy as np